In [1]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-

In [5]:
pip install nltk

In [2]:
# Load library
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, matthews_corrcoef, roc_curve, auc
import pickle
from keras.models import load_model
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle as pk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from keras.layers import TextVectorization
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, Conv1D, MaxPooling1D
from keras.utils import to_categorical
import re
from __future__ import print_function

from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K
from __future__ import print_function
import _pickle as cPickle

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Convolution1D, MaxPooling1D, Dense, Dropout, Activation

from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from keras.models import model_from_json
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import random
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [3]:
def twoTupleDic():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          for jj in AA_list_sort:
            AA_dict[i+j+jj] = numm
        AA_dict[i] = numm
        numm += 1
    return AA_dict

In [4]:
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = "/content/drive/MyDrive/CLW_SUMO/Data/"
path_model = "/content/drive/MyDrive/CLW_SUMO/Model/"
path_result = "/content/drive/MyDrive/CLW_SUMO/Result/"


In [7]:
# Load Traindata
import pandas as pd
file_train = "Train_Sumo.csv"
df_train =pd.read_csv(path+file_train)
k =3
texts_train =[] #PTMsequend kmer
for i in df_train['Sequence']:
  temp = ProSentence(i,k) # Biểu diễn dữ liệu đầu vào thành token Kmer
  texts_train.append(temp)
df_train['k_mer'] =texts_train
df_train


,Protein,Sequence,Label,k_mer
0,Q96GD4,ALKVLFKSQIE,1,ALK LKV KVL VLF LFK FKS KSQ SQI QIE
1,Q9NPH2,RPGPSLKRVGP,1,RPG PGP GPS PSL SLK LKR KRV RVG VGP
2,Q9GZR7,GKWKEVKIDPN,1,GKW KWK WKE KEV EVK VKI KID IDP DPN
3,Q149N8,HTRQDVKQDAL,1,HTR TRQ RQD QDV DVK VKQ KQD QDA DAL
4,P11474,RCLPGHKEEED,1,RCL CLP LPG PGH GHK HKE KEE EEE EED
...,...,...,...,...
14947,P62851,MPPKDDKKKKD,0,MPP PPK PKD KDD DDK DKK KKK KKK KKD
14948,Q8IYM0,RDPKGKKRFIL,0,RDP DPK PKG KGK GKK KKR KRF RFI FIL
14949,Q8NC96,EMDARPKLDLG,0,EMD MDA DAR ARP RPK PKL KLD LDL DLG
14950,S4R3E2,KADPGVKSECL,0,KAD ADP DPG PGV GVK VKS KSE SEC ECL


In [8]:
# Tokenizer train data input Word2vec
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
print(data_token[:2])

[['ALK', 'LKV', 'KVL', 'VLF', 'LFK', 'FKS', 'KSQ', 'SQI', 'QIE'], ['RPG', 'PGP', 'GPS', 'PSL', 'SLK', 'LKR', 'KRV', 'RVG', 'VGP']]


In [9]:
# Len of the K_mer[1]
len(data_token[1])
MAX_SEQUENCE_LENGTH = len(data_token[1])
print(MAX_SEQUENCE_LENGTH)

9


In [14]:
# Build Word2vec on train data
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pickle as cPickle
import pandas as pd
word_index1 = twoTupleDic() # Tham chiếu texts thành word_index1  trong bộ từ điển
sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    sequences.append(each_index_list)

data_sequence = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

data_sequence
tokenizer = Tokenizer(num_words=None) # MAX_NB_WORDS
tokenizer.fit_on_texts(texts_train)
tokenizer.texts_to_sequences(texts_train) #num_words
word_index2 = tokenizer.word_index

#word2vec
EMBEDDING_DIM = 300
name_model_w2vec ="word2vec.model"
w2v_model = Word2Vec(sentences=data_token , vector_size = EMBEDDING_DIM, window=5, min_count=1,sg=0)
w2v_model.train(data_token , total_examples=len(texts_train), epochs=10)
w2v_model.save(path+name_model_w2vec)
Word2VecModel = Word2Vec.load(path_model+name_model_w2vec)

In [16]:
embedding_matrix = np.zeros((len(word_index1)+1, EMBEDDING_DIM))
for word, i in word_index1.items():
    word_lower = word.lower()
    if word_lower in Word2VecModel.wv:
        embedding_vector = Word2VecModel.wv[word_lower]
    else:
        # Gán vector ngẫu nhiên nếu từ không có trong vocab
        embedding_vector = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))
    embedding_matrix[i] = embedding_vector

In [17]:
from sklearn.model_selection import train_test_split
X = data_sequence
y = np.array(df_train['Label'])


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, roc_curve, auc
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, Dense, concatenate

# --- Cấu hình ---
nb_epoch = 100
nb_filter = 128        # sửa từ nb_filte
k = 10
batch_size = 32
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# --- Biến lưu kết quả ---
fold_results = []
all_metrics = []
confusion_matrices = []
all_auc_scores = []
best_auc = 0
best_model = None

plt.figure(figsize=(8, 6))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Training on Fold {fold+1}/{k}...")

    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))

    # Nhánh CNN
    cnn_branch = Embedding(input_dim=len(embedding_matrix),
                           output_dim=EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable=False)(input_layer)
    cnn_branch = Conv1D(filters=nb_filter, kernel_size=3, activation='relu',padding='same')(cnn_branch)
    cnn_branch = MaxPooling1D(pool_size=2)(cnn_branch)
    cnn_branch = Dropout(0.3)(cnn_branch)
    cnn_branch = Conv1D(filters=nb_filter, kernel_size=3, activation='relu',padding='same')(cnn_branch)
    cnn_branch = MaxPooling1D(pool_size=2)(cnn_branch)
    cnn_branch = Flatten()(cnn_branch)

    # Nhánh LSTM
    lstm_branch = Embedding(input_dim=len(embedding_matrix),
                            output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(input_layer)
    lstm_branch = Dropout(0.4)(lstm_branch)
    lstm_branch = LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)(lstm_branch)
    lstm_branch = Flatten()(lstm_branch)

    # Kết hợp
    merged = concatenate([cnn_branch, lstm_branch])
    z = Dense(64, activation='relu')(merged)
    z = Dense(1, activation='sigmoid')(z)

    model = Model(inputs=input_layer, outputs=z)

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',  # Sửa lại phù hợp bài toán nhị phân
                  metrics=['accuracy'])

    model.fit(X_train, Y_train,
              epochs=nb_epoch,
              batch_size=batch_size,
              validation_data=(X_test, Y_test),
              shuffle=True,
              verbose=0)

    # Đánh giá
    results = model.evaluate(X_test, Y_test, verbose=0)
    all_metrics.append(results)

    # Dự đoán
    y_pred_probs = model.predict(X_test)
    y_pred_classes = (y_pred_probs > 0.5).astype(int).flatten()
    y_true_classes = Y_test.flatten()

    # Confusion matrix
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"Fold": fold + 1, "TP": tp, "FP": fp, "TN": tn, "FN": fn})

    print(f"Fold {fold + 1} - TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")

    # ROC-AUC
    fpr, tpr, _ = roc_curve(y_true_classes, y_pred_probs)
    auc_score = auc(fpr, tpr)
    all_auc_scores.append({"Fold": fold + 1, "AUC": auc_score})
    plt.plot(fpr, tpr, label=f"Fold {fold + 1} (AUC = {auc_score:.4f})")

    # Lưu mô hình tốt nhất
    if auc_score > best_auc:
        best_auc = auc_score
        best_model = model

# Lưu mô hình tốt nhất
if best_model:
    best_model.save(path_model + "CLW_SUMO.h5")
    print("\nBest model saved as 'CLW_SUMO.h5'")

# Trung bình kết quả
avg_metrics = np.mean(all_metrics, axis=0)
avg_auc = np.mean([auc_info["AUC"] for auc_info in all_auc_scores])

print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(model.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")
print(f"\nAverage AUC-ROC: {avg_auc:.4f}")

# Lưu Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_auc = pd.DataFrame(all_auc_scores)
with pd.ExcelWriter(path_result + "cv_CLW_SUMO.xlsx") as writer:
    df_confusion.to_excel(writer, sheet_name="Confusion Matrix", index=False)
    df_auc.to_excel(writer, sheet_name="AUC Scores", index=False)
print("\nResults saved to 'cv_CLW_SUMO.xlsx'")

# Vẽ đường cong tổng hợp
plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.5)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve CLW_SUMO (Avg AUC = {avg_auc:.4f})')
plt.legend()
plt.grid()
plt.show()
